# 注意力评分函数
https://zh.d2l.ai/chapter_attention-mechanisms/attention-scoring-functions.html

In [1]:
import math
import torch
from torch import nn
from d2l import torch as d2l

## 1. 隐蔽softmax操作
为了仅将有意义的词元作为值来获取注意力汇聚， 可以指定一个有效序列长度（即词元的个数）， 以便在计算softmax时过滤掉超出指定范围的位置。其中任何超出有效长度的位置都被掩蔽并置为0。

In [2]:
def masked_softmax(X, valid_lens):
    '''通过在最后一个轴上掩蔽元素执行softmax操作'''
    # X: 3D张量，Valid_lens:1D或2D张量
    if valid_lens is None:
        return nn.functional.softmax(X, dim=1)
    else:
        shape = X.shape
        if valid_lens.dim() == 1:
            valid_lens = torch.repeat_interleave(valid_lens, shape[1])
        else:
            valid_lens = valid_lens.reshape(-1)
        # 最后一张上被隐蔽的元素使用一个非常大的负值替换，从而softmax输出为0
        X = d2l.sequence_mask(X.reshape(-1, shape[-1], valid_lens, value=-1e6))
        return nn.functional.softmax(X.reshape(shape), dim=-1)